In [1]:
from src.data_application import DataFrameProcessor
from src.data_local import DataLoader ,MergedDataPreprocessing
from cashe import get_cashed_input

## HJH Dataset
loaded on 23/5/2024

In [2]:
df_original = get_cashed_input()

In [3]:
df_original

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,ELIGIBILITY_API_TRANS_ID,CLAIM_IDENTIFIER,ATTACHED_FILES,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END
0,9521,2024-04-04 22:57:57,230089,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,...,1680350.0,4430078.0,929362-34-4430078_OPDUCAF_2024-04-04_22-57-36_...,FILE,N,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31
1,9521,2024-04-04 23:02:21,230117,202.40,TSH,1.0,220.00,220.00,0.00,220.00,...,1643690.0,4430087.0,1083147-3-4430087_OPDUCAF_2024-04-04_23-01-52_...,FILE,N,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31
2,9521,2024-04-04 23:08:38,230160,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,...,1616370.0,4430098.0,1339075-6-4430098_OPDUCAF_2024-04-04_23-08-22_...,FILE,N,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31
3,9521,2024-04-04 22:26:08,229872,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,...,1638278.0,4430011.0,656928-60-4430011_OPDUCAF_2024-04-04_22-25-48_...,FILE,N,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31
4,9521,2024-04-04 22:34:22,229913,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,...,1657180.0,4430028.0,1019628-25-4430028_OPDUCAF_2024-04-04_22-33-56...,FILE,N,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,...,NaN,4404986.0,1348956-2-4404986_OPDUCAF_2024-01-23_17-17-31_...,FILE,N,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31
488829,10962,2024-01-28 01:01:05,204377,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,...,NaN,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,N,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31
488830,10962,2024-01-28 01:01:05,204377,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,...,NaN,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,N,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31
488831,10962,2024-01-28 01:01:05,204377,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,...,NaN,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,N,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31


In [4]:
preprocessing = MergedDataPreprocessing(df_original)

In [ ]:
df_prep = preprocessing.columns_prep()

In [7]:
df_prep

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,CLAIM_IDENTIFIER,ATTACHED_FILES,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,PatientAgeRange
0,9521,2024-04-04 22:57:57,230089,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,...,4430078.0,929362-34-4430078_OPDUCAF_2024-04-04_22-57-36_...,FILE,0,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31,6
1,9521,2024-04-04 23:02:21,230117,202.40,TSH,1.0,220.00,220.00,0.00,220.00,...,4430087.0,1083147-3-4430087_OPDUCAF_2024-04-04_23-01-52_...,FILE,0,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31,5
2,9521,2024-04-04 23:08:38,230160,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,...,4430098.0,1339075-6-4430098_OPDUCAF_2024-04-04_23-08-22_...,FILE,0,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31,6
3,9521,2024-04-04 22:26:08,229872,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,...,4430011.0,656928-60-4430011_OPDUCAF_2024-04-04_22-25-48_...,FILE,0,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31,6
4,9521,2024-04-04 22:34:22,229913,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,...,4430028.0,1019628-25-4430028_OPDUCAF_2024-04-04_22-33-56...,FILE,0,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,...,4404986.0,1348956-2-4404986_OPDUCAF_2024-01-23_17-17-31_...,FILE,0,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31,8
488829,10962,2024-01-28 01:01:05,204377,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7
488830,10962,2024-01-28 01:01:05,204377,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7
488831,10962,2024-01-28 01:01:05,204377,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7


In [25]:
df_train, df_test = preprocessing.train_test_split(test_size=0.2,random_state=6)

In [26]:
len(df_test)/ len(df_train)

0.2531547822058057

In [12]:
ls1 = list(df_train.sort_values(by='VISIT_ID')['VISIT_ID'])
ls2 = list(df_test.sort_values(by='VISIT_ID')['VISIT_ID'])
intersection = list(set(ls1) & set(ls2))
intersection ## to find the repeated transaction id between them

[]

In [27]:
len(df_train) / len(df_test)

3.9501525165226234

In [29]:
df_train.to_parquet('data/HJH/prq/train.parquet')
df_test.to_parquet('data/HJH/prq/test.parquet')